In [133]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import urllib.request
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [214]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_driver = os.path.join(os.getcwd(), "chromedriver")
driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)

In [215]:
class location:
    def __init__(self,name,lat,long):
        self.name=name
        self.lat=lat
        self.long=long
    
    def get_forecast(self):
        url="https://spotwx.com/products/grib_index.php?model=gem_lam_continental&lat="+str(self.lat)+"&lon="+str(self.long)+"&tz=America/Vancouver&display=table"
        driver.get(url)
        driver.execute_script("$('.buttons-csv').click();")
        time.sleep(2)
        df=pd.read_csv("SpotWx.csv")
        os.remove("SpotWx.csv")
        self.forecast=df
        return df
    
    def weather_synopsis(self,df):
        new_snow=df["SQP"].iloc[-1] #get last accumulated snow
        melting=df["TMP"].max()>0
        wind=df["WS"].max()
        #sun during the day hours only
        cloud=df["CLOUD"].mask((df["TIME"].str.slice(stop=2).astype(int)<8) | (df["TIME"].str.slice(stop=2).astype(int)>20))
        sun=100-cloud.mean()
        data = {'New Snow':  [new_snow],
                'Melting': [melting],
                'Max Winds': [wind],
                'Weighted Sun':[sun]
               }

        result = pd.DataFrame(data, index=[self.name])
        return result

In [216]:
blackcomb=location("black",50.09970, -122.86943)

garibaldi=location("gar",49.93299, -123.03600)

joffre_lake=location("jof",50.34143, -122.47768)

s2s=location("s2s",9.65218, -123.09211)

cloudburst=location("cloud",49.93298, -123.23602)

manning=location("man",49.06805, -120.91646)

coq=location("coq",49.61094, -121.05758)

rogers=location("rog",51.30123, -117.52014)

sites=[blackcomb,garibaldi,joffre_lake,cloudburst,manning,coq,rogers]

In [220]:
view=pd.DataFrame(columns = ['New Snow', 'Melting','Max Winds','Weighted Sun'])

In [221]:
print("Grabbing site data...")
for site in sites:
    print(site.name)
    forecast=site.get_forecast()
    result=site.weather_synopsis(forecast)
    view=view.append(result)

Grabbing site data...
black
gar
jof
cloud
man
coq
rog


In [226]:
view
view.style.background_gradient(cmap="RdYlGn",subset=["New Snow"]) #style snow column

,New Snow,Melting,Max Winds,Weighted Sun
black,0.000000,True,4,100.000000
gar,0.000000,True,12,100.000000
jof,0.000000,True,8,100.000000
cloud,0.000000,True,4,98.703704
man,0.000000,True,9,97.333333
coq,0.000000,True,10,94.296296
rog,0.000000,False,11,98.888889


In [228]:
view.style.background_gradient(cmap="cividis",subset=["Weighted Sun"],vmin=50) #style sun column

,New Snow,Melting,Max Winds,Weighted Sun
black,0.000000,True,4,100.000000
gar,0.000000,True,12,100.000000
jof,0.000000,True,8,100.000000
cloud,0.000000,True,4,98.703704
man,0.000000,True,9,97.333333
coq,0.000000,True,10,94.296296
rog,0.000000,False,11,98.888889


In [98]:
driver.quit()